# 파이썬으로 풀어보는 수학

## 7장 미적분 문제 풀기

### 1.  함수란 무엇인가 ?

`함수`란 입력집합과 출력집합 간의 관계(mapping)를 얘기합니다.
유요한 입력값의 집합을 함수 `도메인(Domain)`이라 하고, 결과집합을 `범위(Range)`라고 합니다.
예를 들어서 $f(x)=\frac {1}{x}$ 라는 함수가 있는 경우 x의 값은 0이 될 수 없습니다.
그러므로 이 함수의 도메인은 0이 아닌 모든 실수와 복소수 입니다.
결과집합인 범위도 0이 아닌 모든 실수와 복소수 입니다.
$f(x)={x}^{2}$에 대해서는 도메인은 모든 양수와 음수이지만, 범위는 양수만 해당합니다.

### 2. 가정(assumptions)

`sympy`의 `Symbol`객체를 이용를 생성한 다음 해당 변수의 값을 가정하고 판단하는 것이 가능합니다.
$x+5$ 값이 0보다 큰지 여부를 확인하는 코드를 작성해 보겠습니다.

In [1]:
from sympy import Symbol

x = Symbol('x')
if x + 5 > 0:
    print('Positive')
else:
    print('Negative')

TypeError: cannot determine truth value of Relational

`TypeError: cannot determine truth value of Relational`라는 오류가 발생합니다.
왜냐면 x의 값이 범위를 알 수 없으므로 해당 식이 0보다 큰지 작은지 추론을 할 수가 없기 때문입니다.
`Symbol` 객체를 `positive=True`로 설정하면 해당 식은 항상 0보다 클 것입니다.

In [2]:
x = Symbol('x', positive=True)
if x + 5 > 0:
    print('Positive')
else:
    print('Negative')

Positive


`negative=True`로 선택하면 참, 거짓이 모두 가능하기 때문에 처음과 같은 `TypeError: cannot determine truth value of Relational` 오류가 발생합니다.

In [3]:
x = Symbol('x', negative=True)
if x + 5 > 0:
    print('Positive')
else:
    print('Negative')

TypeError: cannot determine truth value of Relational

앞에서 살펴본 `positive`, `negative` 외에도 `real`, `integer`, `complex`, `imaginary` 등으로 설정이 가능합니다.

### 3. 함수의 극한

미적분 계산의 기본은 변수의 값이 어떤값에 근접할 때의 함수의 극한값을 찾는 것입니다.
$f(x)=\frac{1}{x}$ 함수의 경우 x가 증가할수록 0에 가까워 집니다.

$\lim _{ x\rightarrow \infty  }{ \frac { 1 }{ x }  } =0$

`sympy`에서는 극한을 `Limit` 객체를 이용해서 생성이 가능합니다.

In [5]:
from sympy import Limit, Symbol, S

x = Symbol('x')
L = Limit(1/x, x, S.Infinity)
L

Limit(1/x, x, oo, dir='-')

극한값 계산에는 `doit()`함수를 사용합니다.

In [6]:
L.doit()

0

0의 방향으로 극한값은 양의 방향에서 0으로 접근을하면 양의 무한대 값이 되며, 음의 방향에서 0으로 접근하면 양의 무한대 값이 됩니다.

In [7]:
Limit(1/x, x, 0, dir='-').doit()

-oo

In [8]:
Limit(1/x, x, 0, dir='+').doit()

oo

정해지지 않은 극한값도 다룰수 있습니다. ( $\frac { 0 }{ 0 }  = \frac { \infty  }{ \infty  } =1$)

In [9]:
from sympy import sin

Limit(sin(x)/x, x, 0).doit()

1

### 4. 연속 복리

이자율 `r`에 대해서 기간 `n` 동안의 복리이자를 계산하는 수식은 다음과 같습니다.

$A={ (1+r) }^{ n }$

여기에서 *r*을 $\frac { 1 }{ n } $로 치환을 한 후 *n*값을 무한대 값으로 극한을 취하면 *e*값에 접근합니다.(${ \left( 1+\frac { 1 }{ n }  \right)  }^{ n }$)

In [3]:
from sympy import Limit, Symbol, symbols, S

n = Symbol('n')
Limit((1+1/n)**n, n, S.Infinity).doit()

E

원금 `P`, 이자율 `r`, 기간 `t`에 대한 복리이자는 다음 공식을 이용합니다.

$A={ P(1+\frac { r }{ n } ) }^{ nt }$

In [6]:
p, r, t = symbols('p,r,t')
A = Limit(p*(1+r/n)**(n*t), n, S.Infinity).doit()
A

p*exp(r*t)

저 수식이 맞는지 계산해보겠습니다.
100만원을 복리 3% 이자 10년동안 예치한 경우 수령액을 계산해 보겠습니다.
저 수식이 맞는지 우리가 일반적으로 알고있는 수식으로 계산한 값과 비교해 보겠습니다.

In [12]:
A.subs({p:1000000, r:0.03, t:10})

1349858.80757600

In [14]:
A2 = p * (1+r)**t
A2.subs({p:1000000, r:0.03, t:10})

1343916.37934412

완벽하게 일치하지는 않지만 큰 차이는 없습니다.

#### 실시간 변화율

어떤 자동차의 이동거리를 계산하는 수식이 $S(t)\quad =\quad 5{ t }^{ 2 }+2t+8$라고 가정을 해봅시다.
시간의 제곱에 비례를 한다는 것을 보면 시간이 지날수록 점점 속도가 빨라지고 있다고 생각할 수 있습니다.

이 함수의 독립변수는 `t`입니다.
특정 시간동안 (`t1`시간에서 `t2`시간까지)의 단위시간당 이동한 거리를 다음 수식으로 계산이 가능합니다.

![수식](./image/DoingMathWithPython.Ch07.equation.02.png)

여기서 `t2`와 `t1`간의 시간차이를 `∂`라 할경우 수식을 다음과 같이 표현이 가능하며,

![수식](./image/DoingMathWithPython.Ch07.equation.03.png)

여기서 `∂`를 0에 근접시키는 극한값은 다음과 같습니다.

![수식](./image/DoingMathWithPython.Ch07.equation.04.png)

`sympy`를 이용해서 극한값을 구해보겠습니다.

In [21]:
t, t1, delta_t = symbols('t,t1,delta_t')

S = 5*t**2 + 2*t + 8
L = (S.subs({t:t1 + delta_t}) - S.subs({t:t1})) / delta_t
Limit(L, delta_t, 0).doit()

10*t1 + 2

극한 계산 결과는 특정시간 `t1`에 대한 `S(t)`의 `변화율`입니다.
즉, `t`에 대한 `가속도`라 할 수 있으며,
`S(t) = 5t^2 + 2t + 8` 함수의 `미분`값이 됩니다.

### 5. 함수의 미분(differential) 계산

함수 `y=f(x)`의 미분은 독립변수 `x`에 대한 종속변수 `y`의 변화율을 계산하는 식입니다.
미분은 `f'(x)`나 `dy/dx`로 표현합니다.
`sympy`의 `Derivative` 객체를 이용하여 미분을 계산 할 수 있습니다.

In [23]:
from sympy import Derivative

D = Derivative(S, t)
D

Derivative(5*t**2 + 2*t + 8, t)

In [25]:
Sd = D.doit()
Sd

10*t + 2

In [26]:
Sd.subs({t:t1})

10*t1 + 2

In [27]:
Sd.subs({t:1})

12

사용자에게 수식을 입력받아서 해당 수식의 미분을 출력하는 프로그램을 작성해 보겠습니다.

In [3]:
from sympy import Derivative, Symbol, sympify, pprint
from sympy.core.sympify import SympifyError

def GetDifferential(f, x): return Derivative(f, x).doit()

def DifferentialCalcuator():    
    f = input('Enter a function :')
    x = input('Enter the variable to differentiate with respect to:')
    try:
        f = sympify(f)
        x = Symbol(x)
    except SympifyError:
        print('Invalid Input')
    else:
        pprint(GetDifferential(f,x))

In [4]:
DifferentialCalcuator()

Enter a function :2*x**2 + 3*x + 1
Enter the variable to differentiate with respect to:x
4⋅x + 3


변수가 2개 이상인 경우에도 정상적으로 동작하는지 확인해 보겠습니다.
다중 변수로 이루어진 함수에서 특정 한 개의 변수만을 대상으로 미분을 하는 것을 `편미분(partial differentiation)`이라 합니다.

In [5]:
DifferentialCalcuator()

Enter a function :2*x**2 + 3*y**2 + 6*x*y + 3
Enter the variable to differentiate with respect to:x
4⋅x + 6⋅y


### 6. 고차 미분과 최대, 최소값 구하기

`Derivative` 클래스를 이용하면 기본적으로는 1차 미분의 결과가 생성됩니다.
고차 미분을 계싼하려면 3번째 인자의 값으로 미분차수를 설정하면 됩니다.

이번에는 특정 함수에서 어떤 구간에서의 최대, 최소값을 계산하기 위해서 1차와 2차 미분을 사용하는 방법을 알아보겠습니다.

```
x^5 - 30x^3 + 50x
```

위 함수에서 `[-5, 5]` 영역에서의 최대값과 최소값을 구해보겠습니다.